Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [1]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive
-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 20476, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 20476 (delta 100), reused 44 (delta 44), pack-reused 20344 (from 4)
Receiving objects: 100% (20476/20476), 63.13 MiB | 13.43 MiB/s, done.
Resolving deltas: 100% (13618/13618), done.
Updating files: 100% (451/451), done.
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.6 MB/s et

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

In [12]:
# ==============================================================================
# STEP 1: Connect to your permanent storage (Google Drive)
# ==============================================================================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ==============================================================================
# STEP 2: Download Models to Google Drive (if they don't exist)
# ==============================================================================
import os

# Define the base path on your Google Drive for all AI assets
drive_base_path = "/content/drive/MyDrive/AI_Manhwa_Studio"
models_path = os.path.join(drive_base_path, "models")

# Define full paths for each model type
checkpoint_path = os.path.join(models_path, "checkpoints")
vae_path = os.path.join(models_path, "vae")
lora_path = os.path.join(models_path, "loras")
controlnet_path = os.path.join(models_path, "controlnet")
upscale_path = os.path.join(models_path, "upscale_models")

# Create directories if they don't exist
for path in [checkpoint_path, vae_path, lora_path, controlnet_path, upscale_path]:
    os.makedirs(path, exist_ok=True)

# Helper function to download files
def download_if_not_exists(url, dest_folder):
    filename = url.split('/')[-1]
    dest_path = os.path.join(dest_folder, filename)
    if not os.path.exists(dest_path):
        print(f"Downloading {filename}...")
        os.system(f"wget -c '{url}' -P '{dest_folder}'")
    else:
        print(f"{filename} already exists. Skipping.")

# --- Download all your models as before ---
download_if_not_exists("https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors", checkpoint_path)
download_if_not_exists("https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors", vae_path)
download_if_not_exists("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", lora_path)
download_if_not_exists("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors", controlnet_path)
download_if_not_exists("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors", controlnet_path)
download_if_not_exists("https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors", controlnet_path)
download_if_not_exists("https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth", upscale_path)
download_if_not_exists("https://huggingface.co/uwg/real-esrgan-x4-anime-6b/resolve/main/real-esrgan-x4-anime-6b.pth", upscale_path)


# ==============================================================================
# STEP 3: Install ComfyUI and its Dependencies
# ==============================================================================
import os
comfyui_path = "/content/ComfyUI"

if not os.path.exists(comfyui_path):
    print("Cloning ComfyUI repository...")
    os.system(f"git clone https://github.com/comfyanonymous/ComfyUI.git {comfyui_path}")

%cd {comfyui_path}
print("Installing dependencies...")
os.system("pip install -r requirements.txt")

# Install the essential ControlNet Preprocessor nodes
preprocessor_path = os.path.join(comfyui_path, "custom_nodes/comfy_controlnet_preprocessors")
if not os.path.exists(preprocessor_path):
    print("Cloning ControlNet Preprocessors...")
    os.system(f"git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors {preprocessor_path}")
    os.system(f"cd {preprocessor_path} && python install.py")
else:
    print("ControlNet Preprocessors already exist.")


# ==============================================================================
# STEP 4: Create the Configuration File to Link Your Drive
# ==============================================================================
import yaml

config_data = {
    'other_ui': {
        'base_path': models_path
    }
}

config_file_path = os.path.join(comfyui_path, "extra_model_paths.yaml")
with open(config_file_path, 'w') as f:
    yaml.dump(config_data, f)
print(f"Created config file at: {config_file_path} with correct format.")


Mounted at /content/drive
animagine-xl-3.1.safetensors already exists. Skipping.
sdxl_vae.safetensors already exists. Skipping.
sd_xl_offset_example-lora_1.0.safetensors already exists. Skipping.
control_v11p_sd15_openpose_fp16.safetensors already exists. Skipping.
control_v11p_sd15s2_lineart_anime_fp16.safetensors already exists. Skipping.
control_v11f1p_sd15_depth_fp16.safetensors already exists. Skipping.
RealESRGAN_x4plus.pth already exists. Skipping.
/content/ComfyUI
Installing dependencies...
Cloning ControlNet Preprocessors...
Created config file at: /content/ComfyUI/extra_model_paths.yaml with correct format.


### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
# ==============================================================================
# CELL 2: LAUNCH COMFYUI AND DISPLAY IN NOTEBOOK
# ==============================================================================
import threading
import time
import socket
from google.colab import output

# This function waits for the ComfyUI server to be ready and then displays it
def iframe_thread(port):
  # Wait until the port is open
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()


  print("to open it in a new window you can open this link here:")
  output.serve_kernel_port_as_window(port)

# Start the thread that will display the UI
# We run this in the background so it can wait for the server to start
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Launch ComfyUI
# The --dont-print-server argument keeps the log clean
# It will run in this cell until you stop it
!python main.py --dont-print-server

Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12978 MB
pytorch version: 2.7.0+cu118
xformers version: 0.0.30
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using xformers attention
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
ComfyUI version: 0.3.41
ComfyUI frontend version: 1.22.2
[Prompt Server] web root: /usr/local/lib/python3.11/dist-packages/comfyui_frontend_package/static

Import times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/websocket_image_save.py

Context impl SQLiteImpl.
Will assume non-transactional DDL.
No target revision found.
to open it in a new window you can open this link here:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

got prompt
invalid prompt: {'type': 'prompt_no_outputs', 'message': 'Prompt has no outputs', 'details': '', 'extra_info': {}}


### Run ComfyUI with localtunnel




In [4]:
!npm install -g localtunnel

import threading

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12978 MB
pytorch version: 2.7.0+cu118
xformers version: 0.0.30
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using xformers attention
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
ComfyUI version: 0.3.41
ComfyUI frontend version: 1.22.2
[Prompt Server] web root: /usr/local/lib/python3.11/dist-packages/comfyui_frontend_package/static

Import times for custom nodes:
   0.0 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/websocket_image_save.py

Context impl SQLiteImpl.
Will assume non-transactional DDL.
No target revision found.

ComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)

The password/enpoint ip for localtunnel is: 35.197.25.86
your url is: https://slimy-windows-camp.loca.lt
got p

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server